# Filtado de mensajes spam

## Descripción del problema real

La recepción de publicidad no deseada a traves mensajes de texto usando SMS (Short Message Service) es un problema que afecta a muchos usuarios de teléfonos móviles. El problema radica en que los usuarios deben pagar por los mesajes recibidos, y por este motivo resulta muy importante que las compañías prestadoras del servicio puedan filtrar mensajes indeseados antes de enviarlos a su destinatario final. Los mensajes tienen una longitud máxima de 160 caracteres, por lo que el texto resulta poco para realizar la clasificación, en comparación con textos más largos (como los emails). Adicionalmente, los errores de digitación dificultan el proceso de detección automática.

## Descripción del problema en términos de los datos

Se tiene una muestra contiene 5574 mensajes en inglés, no codificados y clasificados como legítimos (ham) o spam (http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). La información está almacenada en el archivo `datos/spam-sms.zip`.El problema en términos de los datos consiste en clasificar si un mensaje SMS es legítico o spam, a partir del análisis de las palabras que contiente, partiendo del supuesto de que ciertas palabras que son más frecuentes dependiendo del tipo de mensaje. Esto implica que en la fase de preparación de los datos se deben extraer las palabras que contiene cada mensaje para poder realizar el análsis.

## Aproximaciones posibles

En este caso, se desea comparar los resultados de un modelo de redes neuronales artificiales y otras técnicas estadísticas para realizar la clasificación.

## Requerimientos

Usted debe:

* Preprocesar los datos para representarlos usando bag-of-words.


* Construir un modelo de regresión logística como punto base para la comparación con otros modelos más complejos.


* Construir un modelo de redes neuronales artificiales. Asimismo, debe determinar el número de neuronas en la capa o capas ocultas.


* Utiizar una técnica como crossvalidation u otra similar para establecer la robustez del modelo.


* Presentar métricas de desempeño para establecer las bondades y falencias de cada clasificador.

### Imports

In [2]:
import re 
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix as confusion
from sklearn.metrics import accuracy_score as precision
from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import KFold as KF
from tensorflow import keras
import tensorflow as tf


C:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1)

## Lectura de los datos

In [3]:
datos = pd.DataFrame(columns=['etiqueta','contenido'])

def limpieza(txt):
    return re.sub("\s{1,}"," ",re.sub('[0-9]*', '', re.sub('https?:\/\/.*[\n]*','', txt))).replace('&gt;','').replace('&lt;','').translate(str.maketrans('', '', string.punctuation))

with open('datos/SMSSpamCollection.txt', encoding='utf-8') as abierto:
    lista = str(abierto.read()).split("\n")
    for row in lista:
        patron = "ham\s"
        remplazo = "1,,. "
        row = re.sub(patron, remplazo, row)
        patron = "spam\s"
        remplazo = "0,,. "
        row = re.sub(patron, remplazo, row)
        row = row.split(",,.")
        if(len(row)>1):
            datos = datos.append({
                'etiqueta': row[0],
                'contenido': limpieza(row[1]) 
                }, ignore_index=True) 

### Eliminación de palabras no significativas y creción de la bolsa de palabras

In [4]:
X = datos['contenido']
Y = datos['etiqueta'].astype('int')

stem = nltk.stem.SnowballStemmer('english')

## palabras propias del lenguaje sin aporte al análiss
propias = stopwords.words("english")
def aplicarStem(txt):   
   
    words = [stem.stem(word) for word in txt.split() if word.lower() not in propias]
    return " ".join(words)

ss = []
for i in X:
    ss.append(aplicarStem(i))

# Bolsa de palabras

vectorizador = CountVectorizer(ss)
sX = vectorizador.fit_transform(ss)

sX = sX.toarray()
X_train, X_test, Y_train, Y_test = tts(sX, Y , stratify = Y, test_size = 0.2, random_state=101)


# Regresión Logística

In [5]:
regresion = LogisticRegression()
regresion.fit(X_train, Y_train)
prediccion = regresion.predict(X_test)

print(
      "Matriz de confusión: ", confusion(Y_test, prediccion),
      "\n",
      "Precisión: ",precision(Y_test, prediccion)
)


Matriz de confusión:  [[129  20]
 [  2 964]] 
 Precisión:  0.9802690582959641


# Modelo red neuronal

### Se elige la cantidad de neuronas

In [6]:

nresults = [0,0]
for i in range(1,21):
    ## Crea un modelo vacio    
    model = keras.Sequential()
    ## Adiciona las capas
    model.add(keras.layers.Dense(units = i, activation=tf.nn.relu, input_dim = X_train.shape[1]))
    model.add(keras.layers.Dense(units = 1, activation=tf.nn.sigmoid))
    
    model.summary()
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['acc']
                  )
    ## Estructura del modelo creado
    model.fit(X_train,Y_train,epochs=50,batch_size=30)
    r = model.evaluate(X_test, Y_test)
    if(r[1]>nresults[1]):
        nresults = [i,r[1]]
    model = 0

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 7133      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2         
Total params: 7,135
Trainable params: 7,135
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/50
4459/4459 [==============================] - 1s 138us/sample - loss: 0.6043 - acc: 0.8623
Epoch 2/50
4459/4459 [==============================] - 1s 116us/sample - loss: 0.4358 - acc: 0.8659
Epoch 3/50
4459/4459 [==============================] - 1s 119us/sample - loss: 0.3299 - acc: 0.8659
E

4459/4459 [==============================] - 1s 155us/sample - loss: 0.1111 - acc: 0.9879
Epoch 12/50
4459/4459 [==============================] - 1s 159us/sample - loss: 0.1027 - acc: 0.9913
Epoch 13/50
4459/4459 [==============================] - 1s 167us/sample - loss: 0.0950 - acc: 0.9928s - loss: 0.0957 - acc: 0
Epoch 14/50
4459/4459 [==============================] - 1s 168us/sample - loss: 0.0879 - acc: 0.9939
Epoch 15/50
4459/4459 [==============================] - 1s 171us/sample - loss: 0.0813 - acc: 0.9955
Epoch 16/50
4459/4459 [==============================] - 1s 141us/sample - loss: 0.0753 - acc: 0.9960
Epoch 17/50
4459/4459 [==============================] - 1s 139us/sample - loss: 0.0696 - acc: 0.9964
Epoch 18/50
4459/4459 [==============================] - 1s 144us/sample - loss: 0.0645 - acc: 0.9964
Epoch 19/50
4459/4459 [==============================] - 1s 135us/sample - loss: 0.0598 - acc: 0.9964
Epoch 20/50
4459/4459 [==============================] - 1s 131us/sam

Epoch 34/50
4459/4459 [==============================] - 1s 159us/sample - loss: 0.0034 - acc: 0.9987
Epoch 35/50
4459/4459 [==============================] - 1s 127us/sample - loss: 0.0032 - acc: 0.9987
Epoch 36/50
4459/4459 [==============================] - 1s 125us/sample - loss: 0.0031 - acc: 0.9987
Epoch 37/50
4459/4459 [==============================] - 1s 123us/sample - loss: 0.0029 - acc: 0.9987
Epoch 38/50
4459/4459 [==============================] - 1s 124us/sample - loss: 0.0028 - acc: 0.9987
Epoch 39/50
4459/4459 [==============================] - 1s 146us/sample - loss: 0.0026 - acc: 0.9987
Epoch 40/50
4459/4459 [==============================] - 1s 137us/sample - loss: 0.0025 - acc: 0.9987
Epoch 41/50
4459/4459 [==============================] - 1s 138us/sample - loss: 0.0024 - acc: 0.9987
Epoch 42/50
4459/4459 [==============================] - 1s 137us/sample - loss: 0.0023 - acc: 0.9987
Epoch 43/50
4459/4459 [==============================] - 1s 136us/sample - loss: 0

Epoch 1/50
4459/4459 [==============================] - 1s 155us/sample - loss: 0.5268 - acc: 0.9159
Epoch 2/50
4459/4459 [==============================] - 1s 123us/sample - loss: 0.2620 - acc: 0.9711
Epoch 3/50
4459/4459 [==============================] - 1s 124us/sample - loss: 0.1464 - acc: 0.9812
Epoch 4/50
4459/4459 [==============================] - 1s 117us/sample - loss: 0.0942 - acc: 0.9877
Epoch 5/50
4459/4459 [==============================] - 0s 111us/sample - loss: 0.0666 - acc: 0.9899
Epoch 6/50
4459/4459 [==============================] - 0s 112us/sample - loss: 0.0502 - acc: 0.9924
Epoch 7/50
4459/4459 [==============================] - 1s 115us/sample - loss: 0.0393 - acc: 0.9944
Epoch 8/50
4459/4459 [==============================] - 1s 113us/sample - loss: 0.0318 - acc: 0.9960
Epoch 9/50
4459/4459 [==============================] - 1s 121us/sample - loss: 0.0263 - acc: 0.9960
Epoch 10/50
4459/4459 [==============================] - 1s 125us/sample - loss: 0.0221 - a

4459/4459 [==============================] - 1s 136us/sample - loss: 0.0268 - acc: 0.9996
Epoch 25/50
4459/4459 [==============================] - 1s 135us/sample - loss: 0.0249 - acc: 0.9996
Epoch 26/50
4459/4459 [==============================] - 1s 152us/sample - loss: 0.0231 - acc: 0.9996
Epoch 27/50
4459/4459 [==============================] - 1s 152us/sample - loss: 0.0215 - acc: 0.9996
Epoch 28/50
4459/4459 [==============================] - 1s 136us/sample - loss: 0.0200 - acc: 0.9996
Epoch 29/50
4459/4459 [==============================] - 1s 157us/sample - loss: 0.0186 - acc: 0.9996
Epoch 30/50
4459/4459 [==============================] - 1s 162us/sample - loss: 0.0174 - acc: 0.9996
Epoch 31/50
4459/4459 [==============================] - 1s 167us/sample - loss: 0.0162 - acc: 0.9996
Epoch 32/50
4459/4459 [==============================] - 1s 154us/sample - loss: 0.0151 - acc: 0.9996
Epoch 33/50
4459/4459 [==============================] - 1s 128us/sample - loss: 0.0141 - acc:

...

4459/4459 [==============================] - 1s 141us/sample - loss: 9.3604e-04 - acc: 0.9993
Epoch 48/50
4459/4459 [==============================] - 1s 150us/sample - loss: 8.9585e-04 - acc: 0.9993
Epoch 49/50
4459/4459 [==============================] - 1s 158us/sample - loss: 8.5807e-04 - acc: 0.9993
Epoch 50/50
1115/1115 [==============================] - 0s 195us/sample - loss: 0.1177 - acc: 0.9812
Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 15)                106995    
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 16        
Total params: 107,011
Trainable params: 107,011
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
4459/4459 [==============================] - 1s 184us/sample - loss: 0.4802 - acc: 0.9215


Epoch 13/50
4459/4459 [==============================] - 1s 151us/sample - loss: 0.0065 - acc: 0.9989
Epoch 14/50
4459/4459 [==============================] - 1s 158us/sample - loss: 0.0057 - acc: 0.9991
Epoch 15/50
4459/4459 [==============================] - 1s 187us/sample - loss: 0.0050 - acc: 0.9993
Epoch 16/50
4459/4459 [==============================] - 1s 224us/sample - loss: 0.0045 - acc: 0.9993
Epoch 17/50
4459/4459 [==============================] - 1s 161us/sample - loss: 0.0040 - acc: 0.9993
Epoch 18/50
4459/4459 [==============================] - 1s 152us/sample - loss: 0.0036 - acc: 0.9993
Epoch 19/50
4459/4459 [==============================] - 1s 153us/sample - loss: 0.0033 - acc: 0.9993
Epoch 20/50
4459/4459 [==============================] - 1s 153us/sample - loss: 0.0030 - acc: 0.9993
Epoch 21/50
4459/4459 [==============================] - 1s 158us/sample - loss: 0.0028 - acc: 0.9993
Epoch 22/50
4459/4459 [==============================] - 1s 154us/sample - loss: 0

4459/4459 [==============================] - 1s 157us/sample - loss: 9.0546e-04 - acc: 0.9998
Epoch 36/50
4459/4459 [==============================] - 1s 158us/sample - loss: 8.4253e-04 - acc: 0.9998
Epoch 37/50
4459/4459 [==============================] - 1s 150us/sample - loss: 7.8406e-04 - acc: 0.9998
Epoch 38/50
4459/4459 [==============================] - 1s 146us/sample - loss: 7.2695e-04 - acc: 0.9998
Epoch 39/50
4459/4459 [==============================] - 1s 147us/sample - loss: 6.8127e-04 - acc: 0.9998
Epoch 40/50
4459/4459 [==============================] - 1s 147us/sample - loss: 6.3939e-04 - acc: 0.9998
Epoch 41/50
4459/4459 [==============================] - 1s 147us/sample - loss: 6.0029e-04 - acc: 0.9998
Epoch 42/50
4459/4459 [==============================] - 1s 148us/sample - loss: 5.5968e-04 - acc: 0.9998
Epoch 43/50
4459/4459 [==============================] - 1s 149us/sample - loss: 5.3008e-04 - acc: 0.9998
Epoch 44/50
4459/4459 [==============================] - 1

Epoch 1/50
4459/4459 [==============================] - 1s 220us/sample - loss: 0.4021 - acc: 0.9312
Epoch 2/50
4459/4459 [==============================] - 1s 178us/sample - loss: 0.1289 - acc: 0.9807
Epoch 3/50
4459/4459 [==============================] - 1s 179us/sample - loss: 0.0630 - acc: 0.9901
Epoch 4/50
4459/4459 [==============================] - 1s 188us/sample - loss: 0.0387 - acc: 0.9933
Epoch 5/50
4459/4459 [==============================] - 1s 179us/sample - loss: 0.0266 - acc: 0.9962
Epoch 6/50
4459/4459 [==============================] - 1s 179us/sample - loss: 0.0196 - acc: 0.9962
Epoch 7/50
4459/4459 [==============================] - 1s 176us/sample - loss: 0.0149 - acc: 0.9973
Epoch 8/50
4459/4459 [==============================] - 1s 182us/sample - loss: 0.0118 - acc: 0.9980
Epoch 9/50
4459/4459 [==============================] - 1s 181us/sample - loss: 0.0096 - acc: 0.9982
Epoch 10/50
4459/4459 [==============================] - 1s 211us/sample - loss: 0.0080 - a

4459/4459 [==============================] - 1s 155us/sample - loss: 0.0021 - acc: 0.9993
Epoch 24/50
4459/4459 [==============================] - 1s 154us/sample - loss: 0.0020 - acc: 0.9993
Epoch 25/50
4459/4459 [==============================] - 1s 157us/sample - loss: 0.0018 - acc: 0.9993
Epoch 26/50
4459/4459 [==============================] - 1s 152us/sample - loss: 0.0017 - acc: 0.9993
Epoch 27/50
4459/4459 [==============================] - 1s 155us/sample - loss: 0.0016 - acc: 0.9993
Epoch 28/50
4459/4459 [==============================] - 1s 152us/sample - loss: 0.0014 - acc: 0.9993
Epoch 29/50
4459/4459 [==============================] - 1s 152us/sample - loss: 0.0013 - acc: 0.9993
Epoch 30/50
4459/4459 [==============================] - 1s 166us/sample - loss: 0.0012 - acc: 0.9993
Epoch 31/50
4459/4459 [==============================] - 1s 159us/sample - loss: 0.0011 - acc: 0.9993
Epoch 32/50
4459/4459 [==============================] - 1s 162us/sample - loss: 0.0010 - acc:

### Se elige la cantidad de capas

In [7]:
cresults = [0,0]
for i in range(1,10):
    ## Crea un modelo vacio    
    model = keras.Sequential()
    ## Adiciona las capas
    for j in range(0,i):
        model.add(keras.layers.Dense(units =nresults[0], activation=tf.nn.relu, input_dim = X_train.shape[1]))
    model.add(keras.layers.Dense(units = 1, activation=tf.nn.sigmoid))
    
    model.summary()
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['acc']
                  )
    ## Estructura del modelo creado
    model.fit(X_train,Y_train,epochs=50,batch_size=30)
    r = model.evaluate(X_test, Y_test)
    if(r[1]>cresults[1]):
        cresults = [i,r[1]]
    model = 0

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 8)                 57064     
_________________________________________________________________
dense_41 (Dense)             (None, 1)                 9         
Total params: 57,073
Trainable params: 57,073
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
4459/4459 [==============================] - 1s 193us/sample - loss: 0.4477 - acc: 0.9036
Epoch 2/50
4459/4459 [==============================] - 1s 153us/sample - loss: 0.1890 - acc: 0.9679
Epoch 3/50
4459/4459 [==============================] - 1s 140us/sample - loss: 0.1018 - acc: 0.9845
Epoch 4/50
4459/4459 [==============================] - 1s 140us/sample - loss: 0.0644 - acc: 0.9892
Epoch 5/50
4459/4459 [==============================] - 1s 135us/sample - loss: 0.0450 - acc: 0.9924
Epoch 

4459/4459 [==============================] - 1s 136us/sample - loss: 0.0013 - acc: 1.0000
Epoch 17/50
4459/4459 [==============================] - 1s 134us/sample - loss: 0.0011 - acc: 1.0000
Epoch 18/50
4459/4459 [==============================] - 1s 136us/sample - loss: 9.2287e-04 - acc: 1.0000
Epoch 19/50
4459/4459 [==============================] - 1s 135us/sample - loss: 7.8996e-04 - acc: 1.0000
Epoch 20/50
4459/4459 [==============================] - 1s 134us/sample - loss: 6.6786e-04 - acc: 1.0000
Epoch 21/50
4459/4459 [==============================] - 1s 134us/sample - loss: 5.7552e-04 - acc: 1.0000
Epoch 22/50
4459/4459 [==============================] - 1s 135us/sample - loss: 4.9558e-04 - acc: 1.0000
Epoch 23/50
4459/4459 [==============================] - 1s 136us/sample - loss: 4.3028e-04 - acc: 1.0000
Epoch 24/50
4459/4459 [==============================] - 1s 135us/sample - loss: 3.7600e-04 - acc: 1.0000
Epoch 25/50
4459/4459 [==============================] - 1s 137us/

4459/4459 [==============================] - 1s 143us/sample - loss: 2.8002e-05 - acc: 1.0000
Epoch 35/50
4459/4459 [==============================] - 1s 144us/sample - loss: 2.5139e-05 - acc: 1.0000
Epoch 36/50
4459/4459 [==============================] - 1s 143us/sample - loss: 2.2632e-05 - acc: 1.0000
Epoch 37/50
4459/4459 [==============================] - 1s 141us/sample - loss: 2.0378e-05 - acc: 1.0000
Epoch 38/50
4459/4459 [==============================] - 1s 140us/sample - loss: 1.8337e-05 - acc: 1.0000
Epoch 39/50
4459/4459 [==============================] - 1s 139us/sample - loss: 1.6508e-05 - acc: 1.0000
Epoch 40/50
4459/4459 [==============================] - 1s 137us/sample - loss: 1.4899e-05 - acc: 1.0000
Epoch 41/50
4459/4459 [==============================] - 1s 137us/sample - loss: 1.3466e-05 - acc: 1.0000
Epoch 42/50
4459/4459 [==============================] - 1s 139us/sample - loss: 1.2194e-05 - acc: 1.0000
Epoch 43/50
4459/4459 [==============================] - 1

1115/1115 [==============================] - 0s 222us/sample - loss: 0.2028 - acc: 0.9821
Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_54 (Dense)             (None, 8)                 57064     
_________________________________________________________________
dense_55 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_56 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_57 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_58 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_59 (Dense)             (None, 1)                 9         
Total params: 57,361
Trainabl

4459/4459 [==============================] - 1s 221us/sample - loss: 0.0111 - acc: 0.9973
Epoch 5/50
4459/4459 [==============================] - 1s 227us/sample - loss: 0.0048 - acc: 0.9987
Epoch 6/50
4459/4459 [==============================] - 1s 202us/sample - loss: 0.0026 - acc: 0.9991
Epoch 7/50
4459/4459 [==============================] - 1s 205us/sample - loss: 0.0015 - acc: 0.9996
Epoch 8/50
4459/4459 [==============================] - 1s 236us/sample - loss: 8.7458e-04 - acc: 1.0000
Epoch 9/50
4459/4459 [==============================] - 1s 212us/sample - loss: 5.5652e-04 - acc: 1.0000
Epoch 10/50
4459/4459 [==============================] - 1s 181us/sample - loss: 3.9327e-04 - acc: 1.0000
Epoch 11/50
4459/4459 [==============================] - 1s 175us/sample - loss: 2.8796e-04 - acc: 1.0000
Epoch 12/50
4459/4459 [==============================] - 1s 172us/sample - loss: 2.1152e-04 - acc: 1.0000
Epoch 13/50
4459/4459 [==============================] - 1s 173us/sample - loss

4459/4459 [==============================] - 1s 173us/sample - loss: 3.5454e-05 - acc: 1.0000
Epoch 18/50
4459/4459 [==============================] - 1s 172us/sample - loss: 3.0449e-05 - acc: 1.0000
Epoch 19/50
4459/4459 [==============================] - 1s 176us/sample - loss: 2.6235e-05 - acc: 1.0000
Epoch 20/50
4459/4459 [==============================] - 1s 180us/sample - loss: 2.2785e-05 - acc: 1.0000
Epoch 21/50
4459/4459 [==============================] - 1s 180us/sample - loss: 1.9844e-05 - acc: 1.0000
Epoch 22/50
4459/4459 [==============================] - 1s 182us/sample - loss: 1.7401e-05 - acc: 1.0000s - loss: 1.1587e-0
Epoch 23/50
4459/4459 [==============================] - 1s 175us/sample - loss: 1.5319e-05 - acc: 1.0000
Epoch 24/50
4459/4459 [==============================] - 1s 172us/sample - loss: 1.3488e-05 - acc: 1.0000
Epoch 25/50
4459/4459 [==============================] - 1s 169us/sample - loss: 1.1995e-05 - acc: 1.0000
Epoch 26/50
4459/4459 [================

4459/4459 [==============================] - 1s 177us/sample - loss: 6.7715e-06 - acc: 1.0000
Epoch 30/50
4459/4459 [==============================] - 1s 175us/sample - loss: 6.0624e-06 - acc: 1.0000
Epoch 31/50
4459/4459 [==============================] - 1s 179us/sample - loss: 5.4326e-06 - acc: 1.0000
Epoch 32/50
4459/4459 [==============================] - 1s 176us/sample - loss: 4.8730e-06 - acc: 1.0000
Epoch 33/50
4459/4459 [==============================] - 1s 179us/sample - loss: 4.3713e-06 - acc: 1.0000
Epoch 34/50
4459/4459 [==============================] - 1s 175us/sample - loss: 3.9467e-06 - acc: 1.0000
Epoch 35/50
4459/4459 [==============================] - 1s 177us/sample - loss: 3.5633e-06 - acc: 1.0000
Epoch 36/50
4459/4459 [==============================] - 1s 176us/sample - loss: 3.2184e-06 - acc: 1.0000
Epoch 37/50
4459/4459 [==============================] - 1s 175us/sample - loss: 2.9042e-06 - acc: 1.0000
Epoch 38/50
4459/4459 [==============================] - 1

4459/4459 [==============================] - 1s 180us/sample - loss: 0.0031 - acc: 0.9996
Epoch 42/50
4459/4459 [==============================] - 1s 180us/sample - loss: 0.0030 - acc: 0.9996
Epoch 43/50
4459/4459 [==============================] - 1s 187us/sample - loss: 0.0031 - acc: 0.9996
Epoch 44/50
4459/4459 [==============================] - 1s 183us/sample - loss: 0.0030 - acc: 0.9996
Epoch 45/50
4459/4459 [==============================] - 1s 182us/sample - loss: 0.0030 - acc: 0.9996
Epoch 46/50
4459/4459 [==============================] - 1s 185us/sample - loss: 0.0030 - acc: 0.9996
Epoch 47/50
4459/4459 [==============================] - 1s 182us/sample - loss: 0.0030 - acc: 0.9996
Epoch 48/50
4459/4459 [==============================] - ETA: 0s - loss: 0.0031 - acc: 0.999 - 1s 184us/sample - loss: 0.0030 - acc: 0.9996
Epoch 49/50
4459/4459 [==============================] - 1s 182us/sample - loss: 0.0032 - acc: 0.9996
Epoch 50/50
1115/1115 [==============================] -

## Cantidades óptimas de neuronas y capas

In [8]:
print("Neuronas ---- " ,nresults)
print("Capas ---- " ,cresults)

Neuronas ----  [8, 0.9829596]
Capas ----  [7, 0.9847534]


## Modelo definitivo

In [9]:
model = keras.Sequential()
## Adiciona las capas
for j in range(0,cresults[0]):
    model.add(keras.layers.Dense(units =nresults[0], activation=tf.nn.relu, input_dim = X_train.shape[1]))
model.add(keras.layers.Dense(units = 1, activation=tf.nn.sigmoid))
    
model.summary()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc','mse']
              )
## Estructura del modelo creado
model.fit(X_train,Y_train,epochs=100,batch_size=30)
r = model.evaluate(X_test, Y_test)


Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_94 (Dense)             (None, 8)                 57064     
_________________________________________________________________
dense_95 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_96 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_97 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_98 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_99 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_100 (Dense)            (None, 8)               

4459/4459 [==============================] - 1s 179us/sample - loss: 9.6596e-07 - acc: 1.0000 - mean_squared_error: 5.9750e-11
Epoch 50/100
4459/4459 [==============================] - 1s 179us/sample - loss: 8.8016e-07 - acc: 1.0000 - mean_squared_error: 5.2413e-11
Epoch 51/100
4459/4459 [==============================] - 1s 176us/sample - loss: 8.0213e-07 - acc: 1.0000 - mean_squared_error: 4.1911e-11
Epoch 52/100
4459/4459 [==============================] - 1s 180us/sample - loss: 7.3235e-07 - acc: 1.0000 - mean_squared_error: 3.4699e-11
Epoch 53/100
4459/4459 [==============================] - 1s 178us/sample - loss: 6.6980e-07 - acc: 1.0000 - mean_squared_error: 3.0965e-11
Epoch 54/100
4459/4459 [==============================] - 1s 177us/sample - loss: 6.1344e-07 - acc: 1.0000 - mean_squared_error: 2.4957e-11
Epoch 55/100
4459/4459 [==============================] - 1s 179us/sample - loss: 5.6011e-07 - acc: 1.0000 - mean_squared_error: 2.1644e-11
Epoch 56/100
4459/4459 [=========

In [10]:
print("Precisión del modelo definitivo ", r[1])
print("MSE del modelo definitivo ", r[2])

Precisión del modelo definitivo  0.9829596
MSE del modelo definitivo  0.016154205


## Validación Cruzada

In [11]:
def validacion(indice_entren, indice_test, X, Y):
    
    X_train = X[indice_entren]
    X_test = X[indice_test]
    Y_train = Y.iloc[indice_entren]
    Y_test = Y.iloc[indice_test]
        
    model = keras.Sequential()
    ## Adiciona las capas
    for j in range(0,cresults[0]):
        model.add(keras.layers.Dense(units =nresults[0], activation=tf.nn.relu, input_dim = X_train.shape[1]))
    model.add(keras.layers.Dense(units = 1, activation=tf.nn.sigmoid))
    
    model.summary()
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc','mse']
              )
## Estructura del modelo creado
    model.fit(X_train,Y_train,epochs=100,batch_size=30)
    r = model.evaluate(X_test, Y_test)

            
    return r[2]
    
kf = KF(n_splits=4)

crossval = []
for indice_entren, indice_test in kf.split(X_train):
    crossval.append(validacion(indice_entren,indice_test, X_train, Y_train))

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_102 (Dense)            (None, 8)                 57064     
_________________________________________________________________
dense_103 (Dense)            (None, 8)                 72        
_________________________________________________________________
dense_104 (Dense)            (None, 8)                 72        
_________________________________________________________________
dense_105 (Dense)            (None, 8)                 72        
_________________________________________________________________
dense_106 (Dense)            (None, 8)                 72        
_________________________________________________________________
dense_107 (Dense)            (None, 8)                 72        
_________________________________________________________________
dense_108 (Dense)            (None, 8)               

3344/3344 [==============================] - 1s 181us/sample - loss: 0.0074 - acc: 1.0000 - mean_squared_error: 3.8861e-04
Epoch 53/100
3344/3344 [==============================] - 1s 197us/sample - loss: 0.0071 - acc: 1.0000 - mean_squared_error: 3.5355e-04
Epoch 54/100
3344/3344 [==============================] - 1s 198us/sample - loss: 0.0067 - acc: 1.0000 - mean_squared_error: 3.2181e-04
Epoch 55/100
3344/3344 [==============================] - 1s 185us/sample - loss: 0.0064 - acc: 1.0000 - mean_squared_error: 2.9292e-04
Epoch 56/100
3344/3344 [==============================] - 1s 179us/sample - loss: 0.0061 - acc: 1.0000 - mean_squared_error: 2.6673e-04
Epoch 57/100
3344/3344 [==============================] - 1s 181us/sample - loss: 0.0058 - acc: 1.0000 - mean_squared_error: 2.4284e-04
Epoch 58/100
3344/3344 [==============================] - 1s 179us/sample - loss: 0.0056 - acc: 1.0000 - mean_squared_error: 2.2100e-04
Epoch 59/100
3344/3344 [==============================] - 1s 

Epoch 1/100
3344/3344 [==============================] - 1s 306us/sample - loss: 0.4821 - acc: 0.8669 - mean_squared_error: 0.1586
Epoch 2/100
3344/3344 [==============================] - 1s 210us/sample - loss: 0.1759 - acc: 0.8938 - mean_squared_error: 0.0554
Epoch 3/100
3344/3344 [==============================] - 1s 210us/sample - loss: 0.0624 - acc: 0.9937 - mean_squared_error: 0.0155
Epoch 4/100
3344/3344 [==============================] - 1s 208us/sample - loss: 0.0155 - acc: 0.9970 - mean_squared_error: 0.0030
Epoch 5/100
3344/3344 [==============================] - 1s 202us/sample - loss: 0.0078 - acc: 0.9982 - mean_squared_error: 0.0015
Epoch 6/100
3344/3344 [==============================] - 1s 203us/sample - loss: 0.0050 - acc: 0.9988 - mean_squared_error: 9.7390e-04
Epoch 7/100
3344/3344 [==============================] - 1s 198us/sample - loss: 0.0021 - acc: 0.9997 - mean_squared_error: 3.3788e-04
Epoch 8/100
3344/3344 [==============================] - 1s 200us/sample - 

3344/3344 [==============================] - 1s 184us/sample - loss: 1.2803e-06 - acc: 1.0000 - mean_squared_error: 4.6566e-10
Epoch 59/100
3344/3344 [==============================] - 1s 182us/sample - loss: 1.1703e-06 - acc: 1.0000 - mean_squared_error: 3.7670e-10
Epoch 60/100
3344/3344 [==============================] - 1s 184us/sample - loss: 1.0779e-06 - acc: 1.0000 - mean_squared_error: 3.2626e-10
Epoch 61/100
3344/3344 [==============================] - 1s 181us/sample - loss: 1.0021e-06 - acc: 1.0000 - mean_squared_error: 2.7549e-10
Epoch 62/100
3344/3344 [==============================] - 1s 184us/sample - loss: 9.2386e-07 - acc: 1.0000 - mean_squared_error: 2.3939e-10
Epoch 63/100
3344/3344 [==============================] - 1s 181us/sample - loss: 8.5682e-07 - acc: 1.0000 - mean_squared_error: 2.0491e-10
Epoch 64/100
3344/3344 [==============================] - 1s 183us/sample - loss: 7.9684e-07 - acc: 1.0000 - mean_squared_error: 1.7200e-10
Epoch 65/100
3344/3344 [=========

Epoch 6/100
3344/3344 [==============================] - 1s 208us/sample - loss: 0.0036 - acc: 0.9994 - mean_squared_error: 7.2673e-04
Epoch 7/100
3344/3344 [==============================] - 1s 213us/sample - loss: 0.0022 - acc: 0.9994 - mean_squared_error: 4.5206e-04
Epoch 8/100
3344/3344 [==============================] - 1s 191us/sample - loss: 0.0015 - acc: 0.9994 - mean_squared_error: 3.3553e-04
Epoch 9/100
3344/3344 [==============================] - 1s 186us/sample - loss: 0.0010 - acc: 0.9997 - mean_squared_error: 2.4645e-04
Epoch 10/100
3344/3344 [==============================] - 1s 190us/sample - loss: 7.9447e-04 - acc: 0.9997 - mean_squared_error: 1.6753e-04
Epoch 11/100
3344/3344 [==============================] - 1s 189us/sample - loss: 5.7374e-04 - acc: 0.9997 - mean_squared_error: 1.2139e-04
Epoch 12/100
3344/3344 [==============================] - 1s 190us/sample - loss: 4.5768e-04 - acc: 0.9997 - mean_squared_error: 9.1832e-05
Epoch 13/100
3344/3344 [================

3344/3344 [==============================] - 1s 186us/sample - loss: 1.2408e-06 - acc: 1.0000 - mean_squared_error: 1.1511e-09
Epoch 65/100
3344/3344 [==============================] - 1s 186us/sample - loss: 1.1558e-06 - acc: 1.0000 - mean_squared_error: 1.0156e-09
Epoch 66/100
3344/3344 [==============================] - 1s 188us/sample - loss: 1.0758e-06 - acc: 1.0000 - mean_squared_error: 8.9893e-10
Epoch 67/100
3344/3344 [==============================] - 1s 185us/sample - loss: 1.0103e-06 - acc: 1.0000 - mean_squared_error: 7.9807e-10
Epoch 68/100
3344/3344 [==============================] - 1s 189us/sample - loss: 9.4186e-07 - acc: 1.0000 - mean_squared_error: 6.9774e-10
Epoch 69/100
3344/3344 [==============================] - 1s 192us/sample - loss: 8.7957e-07 - acc: 1.0000 - mean_squared_error: 6.2080e-10
Epoch 70/100
3344/3344 [==============================] - 1s 189us/sample - loss: 8.2391e-07 - acc: 1.0000 - mean_squared_error: 5.4809e-10
Epoch 71/100
3344/3344 [=========

3345/3345 [==============================] - 1s 188us/sample - loss: 2.4457e-04 - acc: 1.0000 - mean_squared_error: 2.6016e-06
Epoch 12/100
3345/3345 [==============================] - 1s 186us/sample - loss: 1.8957e-04 - acc: 1.0000 - mean_squared_error: 1.7466e-06
Epoch 13/100
3345/3345 [==============================] - 1s 198us/sample - loss: 1.4950e-04 - acc: 1.0000 - mean_squared_error: 1.1555e-06
Epoch 14/100
3345/3345 [==============================] - 1s 263us/sample - loss: 1.1920e-04 - acc: 1.0000 - mean_squared_error: 6.6163e-07
Epoch 15/100
3345/3345 [==============================] - 1s 231us/sample - loss: 9.8150e-05 - acc: 1.0000 - mean_squared_error: 4.1866e-07
Epoch 16/100
3345/3345 [==============================] - 1s 195us/sample - loss: 8.1939e-05 - acc: 1.0000 - mean_squared_error: 2.7095e-07
Epoch 17/100
3345/3345 [==============================] - 1s 192us/sample - loss: 6.9495e-05 - acc: 1.0000 - mean_squared_error: 2.1963e-07
Epoch 18/100
3345/3345 [=========

3345/3345 [==============================] - 1s 240us/sample - loss: 5.5200e-07 - acc: 1.0000 - mean_squared_error: 8.2643e-12
Epoch 70/100
3345/3345 [==============================] - 1s 242us/sample - loss: 5.1465e-07 - acc: 1.0000 - mean_squared_error: 7.0882e-12
Epoch 71/100
3345/3345 [==============================] - 1s 232us/sample - loss: 4.8070e-07 - acc: 1.0000 - mean_squared_error: 6.2958e-12
Epoch 72/100
3345/3345 [==============================] - 1s 252us/sample - loss: 4.4834e-07 - acc: 1.0000 - mean_squared_error: 5.4678e-12
Epoch 73/100
3345/3345 [==============================] - 1s 254us/sample - loss: 4.1838e-07 - acc: 1.0000 - mean_squared_error: 4.7773e-12
Epoch 74/100
3345/3345 [==============================] - 1s 253us/sample - loss: 3.9097e-07 - acc: 1.0000 - mean_squared_error: 4.2106e-12
Epoch 75/100
3345/3345 [==============================] - 1s 341us/sample - loss: 3.6497e-07 - acc: 1.0000 - mean_squared_error: 3.6705e-12
Epoch 76/100
3345/3345 [=========

In [12]:
print("")
print("MSE promedio de la validación cruzada: ", sum(crossval)/len(crossval))    


MSE promedio de la validación cruzada:  0.01914374204352498
